In [10]:
#importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
from tqdm import tqdm
from natsort import natsorted

%matplotlib inline

In [11]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                      transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]),
                  ])

In [55]:
screws/dataset"

In [56]:
part_A_train = os.path.join(root,"train","images")
part_A_test = os.path.join(root,"test","images")
path_sets = [part_A_test]

In [57]:
img_paths = []
for path in path_sets:
    for img_path in natsorted(glob.glob(os.path.join(path, "*.jpg"))):
        img_paths.append(img_path)

In [58]:
model = CSRNet()

In [59]:
model = model.cuda()

In [60]:
checkpoint = torch.load('24checkpoint.pth.tar')

In [61]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [53]:
mae = 0
for i in tqdm(range(len(img_paths))):
    img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground-truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
print (mae/len(img_paths))

100%|██████████| 371/371 [02:27<00:00,  2.52it/s]

11.387934617919099


In [ ]:
from matplotlib import cm as c
folder = 'combined_result/screws/avg/'
n = 1
r = 54
mae = 0
total = 0
under = 0
over = 0
for i in img_paths[:53]:
    fig = plt.figure(figsize=(20, 20))
    avg = []
    for loop in range(7):
        plt.subplot(3, 3, loop+1)
        if loop == 0:
            img = transform(Image.open(i).convert('RGB')).cuda()
            
            tempo = h5py.File((i.replace('.jpg','.h5')).replace('images','ground-truth'),'r')
            temp_1 = np.asarray(tempo['density'])
            original_count = np.round(np.sum(temp_1), 3)
            sub_count = np.round(original_count, 3)
            total += original_count
        else:
            fff = i.replace('_'+str(n), '_'+str(r))
            img = transform(Image.open(fff).convert('RGB')).cuda()
            
            tempo = h5py.File((fff.replace('.jpg','.h5')).replace('images','ground-truth'),'r')
            temp_2 = np.asarray(tempo['density'])
            sub_count = np.round(np.sum(temp_2), 3)
            
            r+= 1
        
        output = model(img.unsqueeze(0))
        predicted = np.round(output.detach().cpu().sum().numpy(), 3)
        temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
        
        plt.title('Original:'+str(sub_count)+ ' predicted:' + str(predicted))
        plt.imshow(temp,cmap = c.jet)
        avg.append(predicted)
        
    average_of_all = np.mean(avg)
    
    
    
    error = average_of_all - original_count
    if error > 0:
        over += error
    elif error <0:
        under += abs(error)
    mae += abs(error)
    
    plt.subplot(3, 3, 8)
    plt.title('Original:'+str(original_count)+ ' predicted avg:'+str(np.round(average_of_all, 3)))
    plt.imshow(temp_1, cmap = c.jet)
    
    
    original_image = plt.imread(i)
    plt.subplot(3, 3, 9)
    plt.title('Original Image')
    plt.imshow(original_image)
    

    
    plt.savefig(folder+str(n)+'.jpg', bbox_inches='tight')
    
    plt.close()
    n = n+1
    

print("Out of {} objects {} objects were miss-counted. {} were overcounted and {} were undercounted"
              .format(np.round(total, 3), np.round(mae, 3), np.round(over,3), np.round(under,3)))
print("Mean absolute error ", np.round(mae/53, 3))


In [43]:
from matplotlib import cm as c
folder = 'combined_result/circles/without_avg/'
n = 1
r = 54
mae = 0
total = 0
under = 0
over = 0
for i in img_paths[:53]:
    fig = plt.figure(figsize=(20, 20))
    avg = []
    img = transform(Image.open(i).convert('RGB')).cuda()
    output = model(img.unsqueeze(0))
    #print("Predicted Count : ",np.round(output.detach().cpu().sum().numpy(), 3))
    predicted = np.round(output.detach().cpu().sum().numpy(), 3)
    temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
    plt.subplot(3, 1, 1)
    plt.title(str(predicted))
    plt.imshow(temp,cmap = c.jet)
    
    
    tempo = h5py.File((i.replace('.jpg','.h5')).replace('images','ground-truth'),'r')
    temp_1 = np.asarray(tempo['density'])
    original_count = np.round(np.sum(temp_1), 3)
    total += original_count
    
    error = predicted - original_count
    if error > 0:
        over += error
    elif error <0:
        under += abs(error)
    mae += abs(error)
    
    plt.subplot(3, 1, 2)
    plt.title('Original:'+str(original_count))
    plt.imshow(temp_1, cmap = c.jet)
    
    
    original_image = plt.imread(i)
    plt.subplot(3, 1, 3)
    plt.title('Original Image')
    plt.imshow(original_image)
    

    
    plt.savefig(folder+str(n)+'.jpg', bbox_inches='tight')
    
    plt.close()
    n = n+1
    

print("Out of {} objects {} objects were miss-counted. {} were overcounted and {} were undercounted"
              .format(np.round(total, 3), np.round(mae, 3), np.round(over,3), np.round(under,3)))
print("Mean absolute error ", np.round(mae/53, 3))


Out of 2575.596 objects 148.018 objects were miss-counted. 101.893 were overcounted and 46.125 were undercounted
Mean absolute error  2.793
